# Scraping Maryland Business Licenses with Selenium

Maryland has a [great portal](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp) for searching business licenses, but the only problem is you have to check a box in order to get in.

1. Try to visit [the public search page](https://jportal.mdcourts.gov/license/pbPublicSearch.jsp)
2. Get redirected to a "I agree to this" page. Click that you've read the disclaimer, click Enter the Site.
3. Click "Search License Records" down at the bottom of the page
4. You're now on the search page! From the "Jurisdiction" dropdown, select "Statewide"
5. In the "Trade Name" field, type "Vap%" to try to find vape shops
6. Click "Next" in the bottom right-hand corner to go to the next page
7. Click "Click for detail" to see the details for a specific business license.

That's a lot of stuff! **Let's get to work.**

## Preparation

### When you search for a business license, what URL should Selenium try to visit first?

In [4]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

In [5]:
from bs4 import BeautifulSoup

In [6]:
driver = webdriver.Chrome()

In [7]:
url = "https://jportal.mdcourts.gov/license/index_disclaimer.jsp"

In [8]:
driver.get(url)

**It isn't going to work, though! It's going to redirect to that intro page.** You can use *Incognito mode* to go back through the "Check the box, etc" series of pages.

### How will you identify the checkbox to check it?

Selenium can find elements by:

- name
- Class
- ID
- CSS selector (**ASK ME WHAT THIS IS** if you don't know)
- XPath (**ASK ME WHAT THIS IS** because you definitely don't know)
- Link text
- Partial link text

So in other words, what's unique about this element?

- *TIP: I have a secret awesome way to do this, but you have to ask.*

In [9]:
driver.find_element_by_id("checkbox").click()

### How will you identify the button to select it, or the form to submit it?

Selenium can submit forms by either

- Selecting the form and using `.submit()`, or
- Selecting the button and using `.click()`

You only need to be able to get **one, not both.**

In [10]:
driver.find_element_by_xpath('html/body/table/tbody/tr[7]/td/form/div/input[2]').click()

In [11]:
driver.find_element_by_link_text("SEARCH LICENSE RECORDS").click()

### Once you're on the next page, how will you click the "SEARCH LICENSE RECORDS" link or follow its URL?

In [12]:
select = Select(driver.find_element_by_name('slcJurisdiction'))

### Now you're on the form page. How do you pick "Statewide" for the jurisdiction dropdown?

In [13]:
select.select_by_visible_text('Statewide')

### How do you type "vap%" into the Trade Name field?

In [14]:
driver.find_element_by_id("txtTradeName").send_keys("vap%")

### How do you click the submit button or submit the form?

In [15]:
driver.find_element_by_tag_name("form").submit()

### How can you find and click the 'Next' button on the search results page?

In [16]:
driver.find_element_by_link_text('Next »').click()

# Okay, let's scrape!

### Use Selenium to search for vape shops statewide, and then click through until it reaches the last page.

Don't scrape yet!

In [17]:
while True:
    try:
        driver.find_element_by_link_text('Next »').click()
    except:
        break

### Use Selenium to scrape the first page of search results for vape shops statewide (well, we'll try).

- *TIP: You can find elements/text using Selenium, or use BeautifulSoup with `doc = BeautifulSoup(driver.page_source)`*

I've included some BeautifulSoup code that might be helpful. If you use it, **ask me how I made it.** It's important.

In [18]:
doc = BeautifulSoup(driver.page_source, 'html.parser')

In [19]:
# IF YOU ARE USING BEAUTIFULSOUP, HERE IS SOME SAMPLE CODE
# IF YOU USE THIS CODE, ASK ME HOW I MADE IT. IT'S IMPORTANT.
business_headers = doc.find_all('tr',class_='searchfieldtitle')
len(business_headers)

2

In [20]:
# You'll probably need to find specific tds inside
# of the rows instead of looking at the whole thing.
for header in business_headers:
    rows = header.find_next_siblings('tr')
    print("HEADER is", header.text.strip())
    print("ROW 0 IS", rows[0].text.strip())
    print("ROW 1 IS", rows[1].text.strip())
    print("ROW 2 IS", rows[2].text.strip())
    print("ROW 3 IS", rows[3].text.strip())
    print("----")

HEADER is 31.
VAPE DOJO
Pending *
ROW 0 IS WALKER TRADING COMPANY INC
Lic. Status: Pending
ROW 1 IS 11600 CROSSROAD CIR  STE E
ROW 2 IS BALTIMORE, MD 21220
ROW 3 IS Baltimore County
----
HEADER is 32.
VAPEZ YARDHOUSE
Pending *
ROW 0 IS YARKHOUSE EMPIRE INC
Lic. Status: Pending
ROW 1 IS 3315 PLAZA WAY
ROW 2 IS WALDORF, MD 20603
ROW 3 IS Charles County
----


### Save these into `vape-results.csv`

In [21]:
#to save to a CSV
# 1. make an emtpy list
# 2. every time through the loop, create a dictionary of your data
# 3. add the dictionary to the list
# 4. when the entire loop is over, convert the list to a dataframe
# 5. save that dataframe


### Open `vape-results.csv` to make sure there aren't any extra weird columns

## Use Selenium to scrape ALL pages of results, save the results into `vape-results-all.csv`.